In [3]:
import boto3
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime,date,timedelta
import json
from sqlalchemy import create_engine

In [2]:
s3 = boto3.client('s3')

In [3]:
bucket='bsft-customers'
prefix_1='discoveryplus.com/events/{}/{}/{}/'.format(date.today().year,date.today().month,date.today().day)
prefix_2='discoveryplus.com/events/{}/{}/{}/'.format(date.today().year,date.today().month,date.today().day+1)
df= pd.DataFrame()

In [6]:
paginator = s3.get_paginator('list_objects')
if (datetime.today().hour+4)<=24:
    for result in paginator.paginate(Bucket = bucket, Prefix=prefix_1):
        for file in result.get('Contents', []):
            if (int(file.get('Key').split('.')[-3].split('-')[-2][-6:]) >= 170000) & \
                (int(file.get('Key').split('.')[-3].split('-')[-2][-6:]) < 172000):
                df_to_add=pd.read_json('s3://'+bucket+'/'+file.get('Key'),compression='gzip',lines = True)
                df = pd.concat([df,df_to_add[df_to_add.event=='announcement - email submission']],\
                               sort=False,ignore_index=True)
                if df_to_add[df_to_add.event=='announcement - email submission'].shape[0]!=0:
                    print(file.get('Key'))
else:
    for result in paginator.paginate(Bucket = bucket, Prefix=prefix_2):
        for file in result.get('Contents', []):
            if (int(file.get('Key').split('.')[-3].split('-')[-2][-6:]) >= (datetime.today().hour-21)*10000) & \
                (int(file.get('Key').split('.')[-3].split('-')[-2][-6:]) < (datetime.today().hour-20)*10000):
                df_to_add=pd.read_json('s3://'+bucket+'/'+file.get('Key'),compression='gzip',lines = True)
                df = pd.concat([df,df_to_add[df_to_add.event=='announcement - email submission']],\
                               sort=False,ignore_index=True)

discoveryplus.com/events/2020/10/28/bsft-events-discoveryplus.com-20201028170920-33.json.gz


In [5]:
df

,_bsft_ts,account_uuid,app_name,app_version,browser_platform,browser_type,browser_version,bsft_sdk_version,device_id,device_manufacturer,...,Product Update Notification Opt In,_bsft_flatten_hashes,_bsft_source,custom_event_type,email,event_type,mParticle Source Feed,name,timestamp_ms,unsubscribed
0,NaN,906a7870-47f5-4e2a-8560-e8e4518e898c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,0.0,mparticle,unknown,hd@test.com,custom_event,Announcement Page Email Capture,announcement - email submission,2020-10-28 17:04:23.924,0.0


In [134]:
paginator = s3.get_paginator('list_objects')
if (datetime.today().hour+4)<=24:
    for result in paginator.paginate(Bucket = bucket, Prefix=prefix_1):
        for file in result.get('Contents', []):
            if (int(file.get('Key').split('.')[-3].split('-')[-2][-6:]) >= (datetime.today().hour+3)*10000) & \
                (int(file.get('Key').split('.')[-3].split('-')[-2][-6:]) < (datetime.today().hour+4)*10000):
                df_to_add=pd.read_json('s3://'+bucket+'/'+file.get('Key'),compression='gzip',lines = True)
                df = pd.concat([df,df_to_add[df_to_add.event=='announcement - email submission']],\
                               sort=False,ignore_index=True)
else:
    for result in paginator.paginate(Bucket = bucket, Prefix=prefix_2):
        for file in result.get('Contents', []):
            if (int(file.get('Key').split('.')[-3].split('-')[-2][-6:]) >= (datetime.today().hour-21)*10000) & \
                (int(file.get('Key').split('.')[-3].split('-')[-2][-6:]) < (datetime.today().hour-20)*10000):
                df_to_add=pd.read_json('s3://'+bucket+'/'+file.get('Key'),compression='gzip',lines = True)
                df = pd.concat([df,df_to_add[df_to_add.event=='announcement - email submission']],\
                               sort=False,ignore_index=True)

In [135]:
df.columns

Index(['_bsft_ts', 'account_uuid', 'app_name', 'app_version',
       'browser_platform', 'browser_type', 'browser_version',
       'bsft_sdk_version', 'device_id', 'device_manufacturer', 'device_token',
       'device_type', 'enable_inapp', 'event', 'event_uuid',
       'limit_ad_tracking', 'network_carrier', 'os_name', 'session_uuid',
       'site', 'timestamp', 'timestamp_epoch', 'user_agent', 'user_uuid',
       'build_number', 'bundle_identifier', 'device_idfv',
       'Product Update Notification Opt In', '_bsft_flatten_hashes',
       '_bsft_source', 'custom_event_type', 'email', 'event_type',
       'mParticle Source Feed', 'name', 'timestamp_ms', 'unsubscribed'],
      dtype='object')

In [144]:
df.groupby(['email']).timestamp.min().to_frame().reset_index()

,email,timestamp
0,Sam_Dunn@discovery.com,2020-10-20 10:30:19
1,han_deng@discovery.com,2020-10-20 20:57:37


In [3]:

creds = json.load(open('//Users/hdeng/Downloads/credentials.json'))
redshift_cred = creds['rs_cred']
engine_url = "postgresql://{0}:{1}@{2}:{3}/{4}".format(redshift_cred['user'],
                                                        redshift_cred['pwd'],
                                                        redshift_cred['host'],
                                                        redshift_cred['port'],
                                                        redshift_cred['dbase'])
conn = create_engine(engine_url).connect()

In [149]:
red=pd.read_sql('select distinct email from digital_analytics_dev.blueshift_ap_signup_raw',conn)

,email
0,cassandra_lim@discovery.com
1,pixeltest@mtodemail.com
2,courtney_wright@discovery.com
3,molly_hoffmaster@discovery.com
4,violet_ding@discovery.com


In [158]:
df=df[~df.email.isin(red.email.tolist())][['email','timestamp']]
df['unknown']=''
df['first_name']=''

In [ ]:
# df_pivot=pd.pivot_table(df_fans,index=['id','platform'], columns='metric',values='data',aggfunc=np.sum)
df[['unknown','email','first_name','timestamp']].to_sql('blueshift_ap_signup_raw', conn, index = False ,if_exists = 'append', \
          schema = 'digital_analytics_dev', chunksize = 1000)
conn.close()

In [5]:
import psycopg2
conn2=psycopg2.connect(dbname=redshift_cred['dbase'],
                        host=redshift_cred['host'],
                        port=redshift_cred['port'],
                        user=redshift_cred['user'],
                        password=redshift_cred['pwd'])
cur=conn2.cursor()
conn2.autocommit=True

In [9]:
cur.execute('create table #t as select * from digital_analytics.blueshift_ap_signup_hourly limit 1;')

In [19]:
print(datetime.now(timezone('EST')).strftime('Y-m-d H:i:s'))

Y-m-d H:i:s


In [21]:
dd=datetime.now(timezone('EST'))

In [28]:
from pytz import timezone
query='''insert into #t
        select 
            to_timestamp('{0}', 'yyyy-mm-dd hh24:mi:ss'),
            0'''.format(dd)
print(query)
cur.execute(query)

insert into #t
        select 
            to_timestamp('2020-11-24 10:30:12.175368-05:00', 'yyyy-mm-dd hh24:mi:ss'),
            0


In [29]:
cur.execute('select * from #t;')

In [30]:
cur.fetchall()

[(datetime.datetime(2020, 11, 24, 10, 21, 28, 852543), 0),
 (datetime.datetime(2020, 11, 24, 10, 30, 12, 175368), 0),
 (datetime.datetime(2020, 11, 24, 10, 30, 12), 0),
 (datetime.datetime(2020, 11, 24, 10, 30, 12), 0)]

In [ ]:
cur.execute('''
            insert into digital_analytics.blueshift_ap_signup_hourly
            select distinct
                now() as timestamp,
                count(distinct email) email_countt
            from digital_analytics_dev.blueshift_ap_signup_raw
            where email not like '%@discovery.com%' and 
            email not like '%@motortrend.com%' and
            date(DATEADD('hour',-4,cast(joined_date as timestamp)))=current_date
            group by 1
            ''')

In [ ]:
cur.close()
conn2.close()

In [1]:
a='''{"site":"discoveryplus.com","account_uuid":"906a7870-47f5-4e2a-8560-e8e4518e898c","timestamp":"2020-12-02T14:10:02Z","email":"kyla.cjones@gmail.com","announcement_emailsubmission":"true","unsubscribed":false,"_bsft_source_event_uuid":"b8637fa9-d948-4403-8839-803209e4943a","_bsft_source_event":"announcement - email submission","event_uuid":"13b26790-34a8-11eb-a6e3-0242ac11001d","event":"identify","user_uuid":"f8f2372e-d5a1-40e5-931f-2572550728ef","session_uuid":"3020c2b3-8c74-4370-ba71-60ff66ffa5a6"}'''

In [5]:
df=pd.read_json(a,lines = True)

In [6]:
df.head

<bound method NDFrame.head of                 _bsft_source_event               _bsft_source_event_uuid  \
0  announcement - email submission  b8637fa9-d948-4403-8839-803209e4943a   

                           account_uuid announcement_emailsubmission  \
0  906a7870-47f5-4e2a-8560-e8e4518e898c                         true   

                   email     event                            event_uuid  \
0  kyla.cjones@gmail.com  identify  13b26790-34a8-11eb-a6e3-0242ac11001d   

                           session_uuid               site  \
0  3020c2b3-8c74-4370-ba71-60ff66ffa5a6  discoveryplus.com   

            timestamp  unsubscribed                             user_uuid  
0 2020-12-02 14:10:02         False  f8f2372e-d5a1-40e5-931f-2572550728ef  >